In [1]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"  # Set JAVA_HOME environment variable

In [2]:
# Imports nécessaires
import os
import csv
import io
import glob
import builtins
import zipfile
import json
import gc
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

# PySpark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import DataFrame

# Pour l'affichage
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Liste des pays de l'Union Européenne (27 membres)
EU_COUNTRIES = {
    'AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 
    'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'NL', 
    'PL', 'PT', 'RO', 'SE', 'SI', 'SK'
}

print("Imports réalisés avec succès")

Imports réalisés avec succès


In [3]:
# Configuration Spark optimisée pour MacBook Pro M3 Max (36GB, 14 cœurs)
# Rappels : réserver ~8-10GB pour macOS, utiliser tous les cœurs disponibles
spark = SparkSession.builder \
    .appName("ObRail_GTFS_Analysis") \
    .master("local[14]") \
    .config("spark.driver.memory", "30g") \
    .config("spark.driver.maxResultSize", "6g") \
    .config("spark.sql.shuffle.partitions", "56") \
    .config("spark.default.parallelism", "56") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.memory.fraction", "0.75") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.sql.autoBroadcastJoinThreshold", "20971520") \
    .config("spark.local.dir", "/tmp/spark-temp") \
    .config("spark.ui.showConsoleProgress", "false") \
    .config("spark.sql.session.timeZone", "UTC") \
    .getOrCreate()

# Configuration du niveau de log
spark.sparkContext.setLogLevel("WARN")

print(f"✓ Spark Session créée avec succès")
print(f"  - Version Spark : {spark.version}")
print(f"  - Master : {spark.sparkContext.master}")
print(f"  - Mémoire Driver : 26 GB")
print(f"  - Cœurs utilisés : 14")
print(f"  - Partitions shuffle : 56")
print(f"  - Application ID : {spark.sparkContext.applicationId}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/23 20:31:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/23 20:31:03 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in standalone/kubernetes and LOCAL_DIRS in YARN).


✓ Spark Session créée avec succès
  - Version Spark : 4.1.1
  - Master : local[14]
  - Mémoire Driver : 26 GB
  - Cœurs utilisés : 14
  - Partitions shuffle : 56
  - Application ID : local-1771875063421


In [4]:
# Lecture des fichiers Parquet de la source 'back on track night'

# Chemin vers les fichiers Parquet
parquet_dir = Path("../data/raw/back_on_track/")

# Liste des fichiers Parquet disponibles
parquet_files = list(parquet_dir.glob("*.parquet"))

print(f"Fichiers Parquet trouvés : {parquet_files}")

# Lecture de chaque table dans un dictionnaire
tables = {}
for file in parquet_files:
    table_name = file.stem
    df = spark.read.parquet(str(file))
    tables[table_name] = df
    print(f"✓ Table '{table_name}' chargée : {df.count()} lignes, {len(df.columns)} colonnes")

# Affichage des noms de tables chargées
print("Tables chargées :", list(tables.keys()))

Fichiers Parquet trouvés : [PosixPath('../data/raw/back_on_track/view_ontd_cities.parquet'), PosixPath('../data/raw/back_on_track/runs.parquet'), PosixPath('../data/raw/back_on_track/agencies.parquet'), PosixPath('../data/raw/back_on_track/view_ontd_list.parquet'), PosixPath('../data/raw/back_on_track/NVScriptsProperties.parquet'), PosixPath('../data/raw/back_on_track/calendar.parquet'), PosixPath('../data/raw/back_on_track/DO_NOT_DELETE___AutoCrat_Job_Se.parquet'), PosixPath('../data/raw/back_on_track/view_ontd_details.parquet'), PosixPath('../data/raw/back_on_track/calendar_dates.parquet'), PosixPath('../data/raw/back_on_track/routes.parquet'), PosixPath('../data/raw/back_on_track/queries.parquet'), PosixPath('../data/raw/back_on_track/view_ontd_map.parquet'), PosixPath('../data/raw/back_on_track/routes_inactive.parquet'), PosixPath('../data/raw/back_on_track/view_trip_export.parquet'), PosixPath('../data/raw/back_on_track/trips_inactive.parquet'), PosixPath('../data/raw/back_on_trac

In [5]:
# Exploration sommaire de chaque table chargée
for name, df in tables.items():
    print(f"\n--- Table: {name} ---")
    df.show(5, truncate=False)
    print(f"Colonnes : {df.columns}")
    print(f"Nombre de lignes : {df.count()}")
    print(f"Schéma :")
    df.printSchema()


--- Table: view_ontd_cities ---


+-----------------+-----------------------+------------+--------------+-----------------------+
|stop_id          |stop_cityname_romanized|stop_country|stop_route_ids|stop_cityname_route_ids|
+-----------------+-----------------------+------------+--------------+-----------------------+
|Aachen Hbf       |Aachen                 |DE          |1,240         |1,240                  |
|Aberdeen         |Aberdeen               |UK          |39            |39                     |
|Adjud            |NULL                   |RO          |22,174        |NULL                   |
|Afyon A Çetinkaya|Afyonkarahisar         |TR          |248           |248                    |
|Alba Iulia       |Alba Iulia             |RO          |15,59         |15,59                  |
+-----------------+-----------------------+------------+--------------+-----------------------+
only showing top 5 rows
Colonnes : ['stop_id', 'stop_cityname_romanized', 'stop_country', 'stop_route_ids', 'stop_cityname_route_ids']
N

26/02/23 20:31:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+--------------------+--------------------------------------------+-------------+----------+--------------+--------------------------------------+---------+--------+-----------------------------------------+------------------------+-----------------------------+-------------------------+-------------+----+------------------------+----------------------+---------------------------------+-----------------------------+----------------------------+---------------------+---------------------------------+-----------------------------------+------------------------+---------------------+---------------------------------+-------------------------------+--------------------------+----------------------+
|Job ID        |Job Name            |Template ID                                 |Data Sheet ID|Header Row|First Data Row|File Name                             |File Type|Share As|Folders                                  |Dynamic Folder Reference|Conditionals                 |Mode

In [6]:
# Aperçu global de la base : statistiques et relations entre tables
import pandas as pd

# Récupérer les stats de chaque table
stats = []
for name, df in tables.items():
    row_count = df.count()
    col_count = len(df.columns)
    stats.append({"table": name, "rows": row_count, "columns": col_count})

stats_df = pd.DataFrame(stats)
print("\nRésumé des tables Parquet :")
display(stats_df)

# Affichage des relations potentielles (par nom de colonnes communes)
from collections import defaultdict
column_map = defaultdict(list)
for name, df in tables.items():
    for col in df.columns:
        column_map[col].append(name)

print("\nColonnes communes entre tables :")
for col, table_list in column_map.items():
    if len(table_list) > 1:
        print(f"- {col} : {table_list}")


Résumé des tables Parquet :


,table,rows,columns
0,view_ontd_cities,721,5
1,runs,0,3
2,agencies,30,15
3,view_ontd_list,201,8
4,NVScriptsProperties,6,2
5,calendar,108,10
6,DO_NOT_DELETE___AutoCrat_Job_Se,1,29
7,view_ontd_details,199,59
8,calendar_dates,59,7
9,routes,206,21



Colonnes communes entre tables :
- stop_id : ['view_ontd_cities', 'stops', 'trip_stop']
- stop_cityname_romanized : ['view_ontd_cities', 'stops']
- stop_country : ['view_ontd_cities', 'stops']
- run_id : ['runs', 'queries']
- train_id : ['runs', 'calendar_dates']
- agency_id : ['agencies', 'view_ontd_details', 'routes', 'view_ontd_map', 'routes_inactive', 'view_trip_export', 'trips_inactive', 'trips']
- route_id : ['view_ontd_list', 'view_ontd_details', 'routes', 'view_ontd_map', 'routes_inactive', 'view_trip_export', 'trips_inactive', 'trips']
- route_long_name : ['view_ontd_list', 'view_ontd_details', 'routes', 'view_ontd_map', 'routes_inactive']
- countries : ['view_ontd_list', 'view_ontd_details', 'routes', 'routes_inactive', 'view_trip_export', 'trips_inactive', 'trips']
- source : ['view_ontd_list', 'view_ontd_details', 'routes', 'routes_inactive', 'trips_inactive']
- service_id : ['calendar', 'calendar_dates', 'view_trip_export', 'trips_inactive', 'trips']
- trip_short_name_0 :

In [7]:
# ═══════════════════════════════════════════════════════════
# CONSTRUCTION DU DATAFRAME BOTN — FORMAT GTFS-LIKE
# Un arrêt par ligne, comme df_cleaned
# ═══════════════════════════════════════════════════════════

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

# === UTILITAIRES ===

# Mapping agency_id → nom (pour multi-opérateurs type "CFM/CFR")
_ag_map = {r["agency_id"]: r["agency_name"] for r in 
    tables["agencies"].select("agency_id", "agency_name").collect()
    if r["agency_name"] is not None}

@F.udf(StringType())
def resolve_agency_name(agency_id):
    if agency_id is None: return None
    return " / ".join(_ag_map.get(p, p) for p in agency_id.split("/"))

# Parser un service_id textuel en days_of_week "1010100"
_day_idx = {"Mon":0, "Tue":1, "Wed":2, "Thu":3, "Fri":4, "Sat":5, "Sun":6, "Son":6}

@F.udf(StringType())
def parse_days_of_week(sid):
    if sid is None: return None
    days = [0] * 7
    if "Daily" in sid or "daily" in sid:
        days = [1] * 7
        if "except" in sid.lower():
            tail = sid.lower().split("except")[-1]
            for d, i in _day_idx.items():
                if d.lower() in tail: days[i] = 0
    else:
        for d, i in _day_idx.items():
            if d in sid: days[i] = 1
    r = "".join(str(d) for d in days)
    return r if r != "0000000" else None

# Mapping country → timezone IANA
_tz_map = {
    "FR":"Europe/Paris","DE":"Europe/Berlin","AT":"Europe/Vienna","CH":"Europe/Zurich",
    "IT":"Europe/Rome","ES":"Europe/Madrid","NL":"Europe/Amsterdam","BE":"Europe/Brussels",
    "CZ":"Europe/Prague","PL":"Europe/Warsaw","HU":"Europe/Budapest","SK":"Europe/Bratislava",
    "SI":"Europe/Ljubljana","HR":"Europe/Zagreb","RS":"Europe/Belgrade","RO":"Europe/Bucharest",
    "BG":"Europe/Sofia","GR":"Europe/Athens","SE":"Europe/Stockholm","NO":"Europe/Oslo",
    "FI":"Europe/Helsinki","DK":"Europe/Copenhagen","EE":"Europe/Tallinn","LT":"Europe/Vilnius",
    "LV":"Europe/Riga","PT":"Europe/Lisbon","IE":"Europe/Dublin","GB":"Europe/London",
    "TR":"Europe/Istanbul","UA":"Europe/Kyiv","MD":"Europe/Chisinau","MK":"Europe/Skopje",
    "ME":"Europe/Podgorica","AL":"Europe/Tirane","LU":"Europe/Luxembourg",
}
tz_map_expr = F.create_map([F.lit(x) for pair in _tz_map.items() for x in pair])


# === 1. TRIPS ===
df_botn_trips = (
    tables["trips"]
    .select(
        F.col("route_id").cast("string").alias("route_id"),
        "trip_id", "trip_short_name", "trip_headsign",
        "agency_id", "service_id",
        F.col("distance"),
        F.col("emissions_co2e"), F.col("co2_per_km"),
    )
)

# === 2. TRIP_STOP : dédup + fix stop_sequence ===
df_botn_ts = (
    tables["trip_stop"]
    .where(F.col("trip_id").isNotNull())                  # supprimer lignes fantômes
    .dropDuplicates(["trip_id", "stop_id"])                # branches : garder 1 arrêt/gare/trip
    .withColumn("dep_str",                                 # extraire HH:MM:SS du departure_time
        F.when(F.col("departure_time").cast("string").rlike("^\\d{4}-"),
            F.substring_index(F.col("departure_time").cast("string"), " ", -1)
        ).otherwise(F.col("departure_time").cast("string"))
    )
    .withColumn("arr_str", F.col("arrival_time").cast("string"))
)

# Identifier les trips avec stop_sequence NULL
_trips_null_seq = [r[0] for r in 
    df_botn_ts.where(F.col("stop_sequence").isNull())
    .select("trip_id").distinct().collect()]

# Renuméroter ces trips par séquence d'origine + horaire
df_botn_ts = (
    df_botn_ts
    .withColumn("stop_sequence",
        F.when(
            F.col("trip_id").isin(_trips_null_seq),
            F.row_number().over(
                Window.partitionBy("trip_id").orderBy(
                    F.when(F.col("stop_sequence").isNotNull(), F.col("stop_sequence").cast("int"))
                     .otherwise(F.lit(9999)),
                    F.coalesce("dep_str", "arr_str")
                )
            ) - 1
        ).otherwise(F.col("stop_sequence").cast("int"))
    )
    .select(
        "trip_id", "stop_id", "stop_sequence",
        F.col("arr_str").alias("arrival_time"),
        F.col("dep_str").alias("departure_time"),
    )
)

# === 3. STOPS ===
df_botn_stops = (
    tables["stops"]
    .select(
        "stop_id", "stop_name", "stop_lat",
        F.expr("try_cast(stop_lon as double)").alias("stop_lon"),
        F.when(F.col("stop_country") == "UK", "GB")
         .otherwise(F.col("stop_country")).alias("country"),
    )
)

# === 4. CALENDAR → days_of_week ===
df_botn_cal = (
    tables["calendar"]
    .filter(F.col("service_id").isNotNull() & ~F.col("service_id").startswith("--"))
    .withColumn("days_of_week", F.concat(
        *[F.coalesce(F.col(d).cast("int"), F.lit(0)) 
          for d in ["monday","tuesday","wednesday","thursday","friday","saturday","sunday"]]
    ))
    .select("service_id", "days_of_week", "start_date", "end_date")
)

# === 5. AGENCIES ===
df_botn_ag = tables["agencies"].select("agency_id", "agency_name", "agency_timezone")

# === 6. ASSEMBLAGE ===
df_botn = (
    df_botn_trips
    .join(df_botn_ts, "trip_id", "inner")
    .join(df_botn_stops, "stop_id", "left")
    .join(df_botn_cal, "service_id", "left")
    .join(df_botn_ag, "agency_id", "left")
    # --- Colonnes fixes ---
    .withColumn("source", F.lit("back_on_track_night"))
    .withColumn("route_type", F.lit(105))
    .withColumn("parent_station", F.lit(None).cast("string"))
    .withColumn("route_short_name", F.col("trip_short_name"))
    .withColumn("route_long_name", F.lit(None).cast("string"))
    .withColumn("city", F.lit(None).cast("string"))
    # --- Corrections ---
    # agency_name : résoudre multi-opérateurs
    .withColumn("agency_name",
        F.when(F.col("agency_name").isNull(), resolve_agency_name(F.col("agency_id")))
         .otherwise(F.col("agency_name")))
    # agency_timezone : CET/EET/WET → IANA via pays du premier arrêt
    .withColumn("agency_timezone",
        F.when(F.col("agency_timezone").startswith("Europe/"), F.col("agency_timezone"))
         .otherwise(tz_map_expr[F.col("country")]))
    # days_of_week : parser depuis service_id si calendar non jointé
    .withColumn("days_of_week",
        F.when(F.col("days_of_week").isNull() & F.col("service_id").isNotNull(),
            parse_days_of_week(F.col("service_id")))
         .otherwise(F.col("days_of_week")))
    .withColumn("days_of_week", F.coalesce(F.col("days_of_week"), F.lit("0000000")))
    # --- Renommage émissions ---
    .withColumnRenamed("emissions_co2e", "botn_emissions_co2e")
    .withColumnRenamed("co2_per_km", "botn_co2_per_km")
    # --- Sélection finale ---
    .select(
        "source", "stop_id", "trip_id", "route_id", "service_id",
        "route_type", "route_short_name", "route_long_name",
        "trip_headsign", "trip_short_name",
        "agency_id", "agency_name", "agency_timezone",
        "stop_name", "stop_lat", "stop_lon", "parent_station",
        "arrival_time", "departure_time", "stop_sequence",
        "start_date", "end_date",
        F.col("distance").alias("botn_distance_km"), "days_of_week",
        "city", "country",
        "botn_emissions_co2e", "botn_co2_per_km",
    )
)

df_botn.cache()
print(f"Lignes BOTN : {df_botn.count():,}")
print(f"Trips : {df_botn.select('trip_id').distinct().count()}")
print(f"Gares : {df_botn.select('stop_id').distinct().count()}")
print(f"Doublons (trip_id, stop_seq) : {df_botn.groupBy('trip_id','stop_sequence').count().where(F.col('count')>1).count()}")
print(f"\nNulls :")
for c in ["stop_name","stop_lat","stop_lon","days_of_week","agency_timezone","agency_name"]:
    print(f"  {c}: {df_botn.where(F.col(c).isNull()).count()}")

Lignes BOTN : 4,119
Trips : 408
Gares : 714
Doublons (trip_id, stop_seq) : 788

Nulls :
  stop_name: 17
  stop_lat: 17
  stop_lon: 17
  days_of_week: 0
  agency_timezone: 17
  agency_name: 0


In [8]:
# ═══════════════════════════════════════════════════════════
# ENRICHISSEMENT CITY POUR BOTN + DIAGNOSTIC COUNTRY
# ═══════════════════════════════════════════════════════════

from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pyspark.sql.types as T
import math

# --- 1. Gares uniques BOTN ---
df_gares_botn = (
    df_botn
    .groupBy("source", "stop_id")
    .agg(
        F.first("stop_name").alias("stop_name"),
        F.first("stop_lat").alias("stop_lat"),
        F.first("stop_lon").alias("stop_lon"),
        F.first("country").alias("botn_country"),  # country d'origine BOTN
    )
    .filter(F.col("stop_lat").isNotNull() & F.col("stop_lon").isNotNull())
)
print(f"Gares uniques BOTN : {df_gares_botn.count():,}")
print(f"  dont lat/lon null : {df_botn.select('source','stop_id').distinct().count() - df_gares_botn.count()}")

# --- 2. Charger GeoNames (même référentiel que GTFS) ---
cities_path = "../data/raw/geonames/cities1000.parquet"
df_cities = (
    spark.read.parquet(cities_path)
    .select("geonameid", "name", "asciiname", "country_code", 
            "latitude", "longitude", "population")
)

# --- 3. Bucketing ---
BUCKET_SIZE = 0.5

df_gares_botn = (
    df_gares_botn
    .withColumn("lat_bucket", F.floor(F.col("stop_lat") / BUCKET_SIZE))
    .withColumn("lon_bucket", F.floor(F.col("stop_lon") / BUCKET_SIZE))
)
df_cities = (
    df_cities
    .withColumn("lat_bucket", F.floor(F.col("latitude") / BUCKET_SIZE))
    .withColumn("lon_bucket", F.floor(F.col("longitude") / BUCKET_SIZE))
)

from itertools import product

df_gares_botn = df_gares_botn.withColumn(
    "buckets",
    F.array(*[
        F.struct(
            (F.col("lat_bucket") + F.lit(dlat)).alias("lat_bucket"),
            (F.col("lon_bucket") + F.lit(dlon)).alias("lon_bucket")
        )
        for dlat, dlon in product([-1, 0, 1], repeat=2)
    ])
)

df_gares_botn_exp = (
    df_gares_botn
    .withColumn("bucket", F.explode("buckets"))
    .withColumn("lat_bucket", F.col("bucket.lat_bucket"))
    .withColumn("lon_bucket", F.col("bucket.lon_bucket"))
    .drop("bucket", "buckets")
)

# --- 4. Haversine + jointure ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))

haversine_udf = F.udf(haversine, T.DoubleType())

df_cities_buckets = df_cities.select(
    "geonameid", "name", "asciiname", "country_code", "latitude", "longitude", "population",
    F.col("lat_bucket").alias("city_lat_bucket"),
    F.col("lon_bucket").alias("city_lon_bucket")
)

df_match = (
    df_gares_botn_exp
    .join(
        F.broadcast(df_cities_buckets),
        (F.col("lat_bucket") == F.col("city_lat_bucket")) &
        (F.col("lon_bucket") == F.col("city_lon_bucket")),
        how="inner"
    )
    .withColumn("distance_km", haversine_udf(
        F.col("stop_lat"), F.col("stop_lon"),
        F.col("latitude"), F.col("longitude")
    ))
    .filter(F.col("distance_km") <= 15)
)

w = Window.partitionBy("source", "stop_id").orderBy(
    F.col("distance_km"), F.col("population").desc()
)
df_match_best = (
    df_match
    .withColumn("rank", F.row_number().over(w))
    .filter(F.col("rank") == 1)
    .select(
        "source", "stop_id",
        F.col("name").alias("city_geo"),
        F.col("country_code").alias("country_geo"),
        "distance_km",
        "botn_country",
    )
)

# --- 5. Statistiques de couverture ---
total = df_gares_botn.count()
matched = df_match_best.count()
print(f"\nGares matchées : {matched:,} / {total:,} ({matched/total:.1%})")
print(f"Gares sans ville (<15km) : {total - matched:,}")

# --- 6. Diagnostic country : BOTN vs GeoNames ---
print("\n" + "=" * 70)
print("DIAGNOSTIC COUNTRY : BOTN déclaré vs GeoNames prédit")
print("=" * 70)

df_country_cmp = (
    df_match_best
    .withColumn("match_country", F.col("botn_country") == F.col("country_geo"))
)

n_match = df_country_cmp.where(F.col("match_country")).count()
n_mismatch = df_country_cmp.where(~F.col("match_country")).count()
n_null_botn = df_country_cmp.where(F.col("botn_country").isNull()).count()

print(f"Country concordant   : {n_match:,}")
print(f"Country discordant   : {n_mismatch:,}")
print(f"Country BOTN null    : {n_null_botn:,}")

# Détail des discordances
if n_mismatch > 0:
    print(f"\n--- Discordances (BOTN → GeoNames) ---")
    (
        df_country_cmp
        .where(~F.col("match_country") & F.col("botn_country").isNotNull())
        .select("stop_id", "botn_country", "country_geo", "city_geo", "distance_km")
        .orderBy("botn_country", "stop_id")
        .show(50, truncate=False)
    )

# --- 7. Appliquer city et country (GeoNames) sur df_botn ---
df_botn_city_map = df_match_best.select(
    "source", "stop_id",
    F.col("city_geo").alias("city"),
    F.col("country_geo").alias("country_geo"),
)

# Remplacer city (null → GeoNames) et country (BOTN → GeoNames si dispo)
df_botn = (
    df_botn.drop("city")
    .join(df_botn_city_map, ["source", "stop_id"], "left")
    .withColumn("country", 
        F.coalesce(F.col("country_geo"), F.col("country"))
    )
    .drop("country_geo")
)

df_botn.cache()
print(f"\ndf_botn enrichi : {df_botn.count():,} lignes")
print(f"City null : {df_botn.where(F.col('city').isNull()).count():,}")
print(f"Country null : {df_botn.where(F.col('country').isNull()).count():,}")

Gares uniques BOTN : 698
  dont lat/lon null : 16

Gares matchées : 693 / 698 (99.3%)
Gares sans ville (<15km) : 5

DIAGNOSTIC COUNTRY : BOTN déclaré vs GeoNames prédit
Country concordant   : 670
Country discordant   : 23
Country BOTN null    : 0

--- Discordances (BOTN → GeoNames) ---
+----------------------------------+------------+-----------+--------------------+---------------------+
|stop_id                           |botn_country|country_geo|city_geo            |distance_km          |
+----------------------------------+------------+-----------+--------------------+---------------------+
|Rankweil                          |AT          |CZ         |Jince               |2.594626443999345    |
|Schladming                        |AT          |CH         |Appenzell           |0.8570236837537698   |
|Schwarzach-St.Veit                |AT          |CZ         |Senice na Hané      |1.2245466069391224E-4|
|St. Valentin                      |AT          |DK         |Buddinge            |1

In [9]:
# 1. Normalisation UK → GB dans BOTN (country d'origine)
df_botn = df_botn.withColumn("country",
    F.when(F.col("country") == "UK", "GB").otherwise(F.col("country"))
)

# 2. Récupérer le country BOTN d'origine pour comparaison
df_botn_country_ref = (
    tables["stops"]
    .select(
        "stop_id",
        F.when(F.col("stop_country") == "UK", "GB")
         .otherwise(F.col("stop_country"))
         .alias("botn_country_orig")
    )
)

# 3. Identifier et supprimer les dissonances
n_before = df_botn.count()

df_botn = (
    df_botn
    .join(df_botn_country_ref, "stop_id", "left")
    .where(
        F.col("country").isNull() |           # pas de GeoNames match → on garde
        F.col("botn_country_orig").isNull() |  # pas de country BOTN → on garde
        (F.col("country") == F.col("botn_country_orig"))  # concordant → on garde
    )
    .drop("botn_country_orig")
)

n_after = df_botn.count()
print(f"Avant : {n_before:,} lignes")
print(f"Après : {n_after:,} lignes")
print(f"Supprimées (dissonances) : {n_before - n_after:,} lignes")
print(f"Trips restants : {df_botn.select('trip_id').distinct().count()}")

Avant : 4,119 lignes
Après : 5,618 lignes
Supprimées (dissonances) : -1,499 lignes
Trips restants : 408


In [10]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : stop_id
# ═══════════════════════════════════════════════════════════

col = "stop_id"
total = df_botn.count()

# Nulls / vides
n_null = df_botn.where(F.col(col).isNull()).count()
n_empty = df_botn.where(F.col(col) == "").count()
n_distinct = df_botn.select(col).distinct().count()

print(f"=== {col} ===")
print(f"Total lignes  : {total:,}")
print(f"Nulls         : {n_null:,}")
print(f"Vides         : {n_empty:,}")
print(f"Distincts     : {n_distinct:,}")

# Format : quels patterns ?
print(f"\n--- Échantillon valeurs ---")
df_botn.select(col).distinct().orderBy(F.rand(42)).show(20, truncate=False)

# Longueur des valeurs
df_botn.select(
    F.min(F.length(col)).alias("len_min"),
    F.max(F.length(col)).alias("len_max"),
    F.mean(F.length(col)).alias("len_avg"),
).show()

# Doublons de casse (ex: "Wien Hbf" vs "wien hbf")
df_case_dup = (
    df_botn.select(col)
    .distinct()
    .withColumn("lower", F.lower(col))
    .groupBy("lower")
    .agg(
        F.count("*").alias("n_variants"),
        F.collect_set(col).alias("variants")
    )
    .where(F.col("n_variants") > 1)
)
print(f"\nDoublons de casse : {df_case_dup.count()}")
if df_case_dup.count() > 0:
    df_case_dup.show(20, truncate=False)

# Caractères spéciaux / encoding suspect
df_botn.select(col).where(
    F.col(col).rlike("[^\x20-\x7E\u00C0-\u024F\u0400-\u04FF]")
).distinct().show(20, truncate=False)

=== stop_id ===
Total lignes  : 5,618
Nulls         : 0
Vides         : 0
Distincts     : 695

--- Échantillon valeurs ---
+-----------------+
|stop_id          |
+-----------------+
|Győr             |
|Koblenz Hbf      |
|Piešťany         |
|Passau Hbf       |
|Budapest-Keleti  |
|Trenčianska Teplá|
|Bârlad           |
|Roma Tiburtina   |
|Bodø             |
|Zaporizhzhia 1   |
|Carei            |
|Katunitsa        |
|Rebrovo          |
|Satu Mare        |
|Oborishte        |
|Caransebeș       |
|Suvorovo         |
|Buzău            |
|Gap              |
|Wörgl Hbf        |
+-----------------+
only showing top 20 rows
+-------+-------+------------------+
|len_min|len_max|           len_avg|
+-------+-------+------------------+
|      3|     35|10.483624065503738|
+-------+-------+------------------+


Doublons de casse : 0
+-------+
|stop_id|
+-------+
+-------+



In [11]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : trip_id
# ═══════════════════════════════════════════════════════════

col = "trip_id"
total = df_botn.count()

n_null = df_botn.where(F.col(col).isNull()).count()
n_empty = df_botn.where(F.col(col) == "").count()
n_distinct = df_botn.select(col).distinct().count()

print(f"=== {col} ===")
print(f"Total lignes  : {total:,}")
print(f"Nulls         : {n_null:,}")
print(f"Vides         : {n_empty:,}")
print(f"Distincts     : {n_distinct:,}")

# Combien d'arrêts par trip ?
df_stops_per_trip = (
    df_botn.groupBy(col).count()
    .select(
        F.min("count").alias("min_stops"),
        F.max("count").alias("max_stops"),
        F.mean("count").alias("avg_stops"),
        F.percentile_approx("count", 0.5).alias("median_stops"),
    )
)
print("\n--- Arrêts par trip ---")
df_stops_per_trip.show()

# Trips avec 1 seul arrêt ?
n_single = df_botn.groupBy(col).count().where(F.col("count") == 1).count()
print(f"Trips à 1 seul arrêt : {n_single}")

# Format des trip_id
print("\n--- Échantillon ---")
df_botn.select(col).distinct().orderBy(F.rand(42)).show(15, truncate=False)

# Doublons de casse
df_case_dup = (
    df_botn.select(col).distinct()
    .withColumn("lower", F.lower(col))
    .groupBy("lower")
    .agg(F.count("*").alias("n"), F.collect_set(col).alias("variants"))
    .where(F.col("n") > 1)
)
print(f"Doublons de casse : {df_case_dup.count()}")
if df_case_dup.count() > 0:
    df_case_dup.show(10, truncate=False)

=== trip_id ===
Total lignes  : 5,618
Nulls         : 0
Vides         : 0
Distincts     : 408

--- Arrêts par trip ---
+---------+---------+------------------+------------+
|min_stops|max_stops|         avg_stops|median_stops|
+---------+---------+------------------+------------+
|        2|       48|13.769607843137255|          13|
+---------+---------+------------------+------------+

Trips à 1 seul arrêt : 0

--- Échantillon ---
+-----------------+
|trip_id          |
+-----------------+
|CFR 1822         |
|ÖBB NJ 408       |
|SJ 1             |
|UZ 194 P         |
|ÖBB NJ 295       |
|FS IC Notte 1958 |
|MÁV IC 1204      |
|SNCF IC Nuit 5737|
|UZ 051           |
|HŽPP/ÖBB EN 40414|
|UEX 333/1373     |
|UZ 003           |
|UZ 042           |
|UZ 050           |
|CFR 1954         |
+-----------------+
only showing top 15 rows
Doublons de casse : 0


In [12]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : route_id
# ═══════════════════════════════════════════════════════════

col = "route_id"
total = df_botn.count()

n_null = df_botn.where(F.col(col).isNull()).count()
n_empty = df_botn.where(F.col(col) == "").count()
n_distinct = df_botn.select(col).distinct().count()

print(f"=== {col} ===")
print(f"Total lignes  : {total:,}")
print(f"Nulls         : {n_null:,}")
print(f"Vides         : {n_empty:,}")
print(f"Distincts     : {n_distinct:,}")

# Trips par route (une route = 2 directions normalement)
df_trips_per_route = (
    df_botn.select("route_id", "trip_id").distinct()
    .groupBy("route_id").count()
    .select(
        F.min("count").alias("min_trips"),
        F.max("count").alias("max_trips"),
        F.mean("count").alias("avg_trips"),
    )
)
print("\n--- Trips par route ---")
df_trips_per_route.show()

# Routes avec != 2 trips (anomalies ?)
df_anom = (
    df_botn.select("route_id", "trip_id").distinct()
    .groupBy("route_id").count()
    .where(F.col("count") != 2)
)
n_anom = df_anom.count()
print(f"Routes avec != 2 trips : {n_anom}")
if n_anom > 0:
    df_anom.orderBy("count").show(20)

# Échantillon
print("\n--- Échantillon ---")
df_botn.select(col).distinct().orderBy(col).show(15, truncate=False)

=== route_id ===
Total lignes  : 5,618
Nulls         : 0
Vides         : 0
Distincts     : 204

--- Trips par route ---
+---------+---------+---------+
|min_trips|max_trips|avg_trips|
+---------+---------+---------+
|        2|        2|      2.0|
+---------+---------+---------+

Routes avec != 2 trips : 0

--- Échantillon ---
+--------+
|route_id|
+--------+
|1       |
|10      |
|100     |
|101     |
|102     |
|103     |
|104     |
|105     |
|106     |
|107     |
|108     |
|109     |
|11      |
|110     |
|111     |
+--------+
only showing top 15 rows


In [13]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : service_id
# ═══════════════════════════════════════════════════════════

col = "service_id"
total = df_botn.count()

n_null = df_botn.where(F.col(col).isNull()).count()
n_empty = df_botn.where(F.col(col) == "").count()
n_distinct = df_botn.select(col).distinct().count()

print(f"=== {col} ===")
print(f"Total lignes  : {total:,}")
print(f"Nulls         : {n_null:,}")
print(f"Vides         : {n_empty:,}")
print(f"Distincts     : {n_distinct:,}")

# Toutes les valeurs distinctes (peu nombreuses attendues)
print("\n--- Valeurs distinctes + fréquence ---")
(
    df_botn.groupBy(col).count()
    .orderBy(F.desc("count"))
    .show(50, truncate=False)
)

# Vérifier la jointure calendar : days_of_week null ?
n_dow_null = df_botn.where(F.col("days_of_week").isNull()).count()
print(f"\ndays_of_week null (service_id non jointé au calendar) : {n_dow_null:,}")

# Quels service_id n'ont pas matché ?
if n_dow_null > 0:
    (
        df_botn.where(F.col("days_of_week").isNull())
        .select(col).distinct()
        .show(20, truncate=False)
    )

=== service_id ===
Total lignes  : 5,618
Nulls         : 50
Vides         : 0
Distincts     : 54

--- Valeurs distinctes + fréquence ---
+---------------------------------------------------+-----+
|service_id                                         |count|
+---------------------------------------------------+-----+
|Daily                                              |2997 |
|2026: Daily                                        |600  |
|2025: Daily                                        |581  |
|2024: every 2nd day                                |226  |
|0-2 days/week (May-Oct)                            |86   |
|2026/06/11-09/06: Daily                            |69   |
|Daily (Jul-Aug)                                    |68   |
|2024: Daily except Sat                             |60   |
|2024/06/15-2024/09/08: Daily                       |58   |
|2024/06/14-2024/09/07: Daily                       |54   |
|2025: Fri, Sun                                     |50   |
|NULL                  

In [14]:
# ═══════════════════════════════════════════════════════════
# DIAGNOSTIC + CORRECTION service_id / days_of_week
# ═══════════════════════════════════════════════════════════

# 1. Les 50 lignes avec service_id NULL — quels trips ?
print("--- Trips avec service_id NULL ---")
(
    df_botn.where(F.col("service_id").isNull())
    .select("trip_id", "route_id", "agency_id", "stop_id")
    .distinct()
    .orderBy("trip_id")
    .show(20, truncate=False)
)

# Combien de trips concernés ?
n_trips_null = df_botn.where(F.col("service_id").isNull()).select("trip_id").distinct().count()
print(f"Trips avec service_id NULL : {n_trips_null}")

# 2. "Fri (Oct-Jun)" : on peut dériver days_of_week manuellement
# Fri = 0000100, période Oct-Jun (saisonnier)
print("\n--- Trips 'Fri (Oct-Jun)' ---")
(
    df_botn.where(F.col("service_id") == "Fri (Oct-Jun)")
    .select("trip_id", "agency_id")
    .distinct()
    .show(10, truncate=False)
)

# 3. Vérifier aussi "Sun (Oct-Jun)" qui a matché au calendar ?
print("\n--- Vérif 'Sun (Oct-Jun)' days_of_week ---")
(
    df_botn.where(F.col("service_id") == "Sun (Oct-Jun)")
    .select("service_id", "days_of_week")
    .distinct()
    .show()
)

# 4. Correction : remplir les days_of_week manquants
# On parse le texte du service_id pour en déduire le pattern
day_map = {
    "Mon": 0, "Tue": 1, "Wed": 2, "Thu": 3, 
    "Fri": 4, "Sat": 5, "Sun": 6, "Son": 6  # "Son" = Sonntag (allemand)
}

# UDF pour parser un service_id texte en days_of_week
from pyspark.sql.types import StringType

def parse_days_of_week(service_id):
    if service_id is None:
        return None
    days = [0] * 7
    # "Daily" ou "daily" = tous les jours
    if "Daily" in service_id or "daily" in service_id:
        # Vérifier "except"
        if "except" in service_id.lower():
            days = [1] * 7
            for day, idx in day_map.items():
                if day.lower() in service_id.lower().split("except")[-1].lower():
                    days[idx] = 0
        else:
            days = [1] * 7
    else:
        for day, idx in day_map.items():
            if day in service_id:
                days[idx] = 1
    result = "".join(str(d) for d in days)
    # Si tout est 0 et pas "Daily", c'est un pattern non reconnu
    return result if result != "0000000" or "Daily" in service_id else None

parse_dow_udf = F.udf(parse_days_of_week, StringType())

# Appliquer seulement où days_of_week est null
df_botn = df_botn.withColumn("days_of_week",
    F.when(F.col("days_of_week").isNull() & F.col("service_id").isNotNull(),
        parse_dow_udf(F.col("service_id"))
    ).otherwise(F.col("days_of_week"))
)

# Remplir les service_id NULL → days_of_week "0000000"
df_botn = df_botn.withColumn("days_of_week",
    F.when(F.col("days_of_week").isNull(), F.lit("0000000"))
     .otherwise(F.col("days_of_week"))
)

n_null = df_botn.where(F.col("days_of_week").isNull()).count()
print(f"days_of_week null : {n_null}")

# Vérifier le résultat
n_still_null = df_botn.where(F.col("days_of_week").isNull()).count()
print(f"\ndays_of_week encore null : {n_still_null} (= service_id NULL)")

# Vérification des corrections
print("\n--- days_of_week corrigés ---")
(
    df_botn.where(F.col("service_id").isin("Fri (Oct-Jun)", "Sun (Oct-Jun)"))
    .select("service_id", "days_of_week")
    .distinct()
    .show()
)

--- Trips avec service_id NULL ---
+-------+--------+---------+------------------+
|trip_id|route_id|agency_id|stop_id           |
+-------+--------+---------+------------------+
|ES 454 |239     |ES       |Köln Süd          |
|ES 454 |239     |ES       |Utrecht Centraal  |
|ES 454 |239     |ES       |Basel SBB         |
|ES 454 |239     |ES       |Duisburg Hbf      |
|ES 454 |239     |ES       |Bern              |
|ES 454 |239     |ES       |Düsseldorf Hbf    |
|ES 454 |239     |ES       |Arnhem Centraal   |
|ES 454 |239     |ES       |Milano Centrale   |
|ES 454 |239     |ES       |Brig              |
|ES 454 |239     |ES       |Amsterdam Centraal|
|ES 455 |239     |ES       |Köln Süd          |
|ES 455 |239     |ES       |Utrecht Centraal  |
|ES 455 |239     |ES       |Basel SBB         |
|ES 455 |239     |ES       |Duisburg Hbf      |
|ES 455 |239     |ES       |Bern              |
|ES 455 |239     |ES       |Düsseldorf Hbf    |
|ES 455 |239     |ES       |Arnhem Centraal   |
|ES 4

In [15]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : route_type
# ═══════════════════════════════════════════════════════════

col = "route_type"
print(f"=== {col} ===")
print(f"Nulls   : {df_botn.where(F.col(col).isNull()).count()}")
print("Valeurs :")
df_botn.groupBy(col).count().show()

=== route_type ===
Nulls   : 0
Valeurs :
+----------+-----+
|route_type|count|
+----------+-----+
|       105| 5618|
+----------+-----+



In [16]:
df_botn = df_botn.withColumn("route_type", F.lit(105))
print("route_type → 105 (Sleeper Rail Service)")

route_type → 105 (Sleeper Rail Service)


In [17]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : route_short_name
# ═══════════════════════════════════════════════════════════

col = "route_short_name"
total = df_botn.count()

n_null = df_botn.where(F.col(col).isNull()).count()
n_empty = df_botn.where(F.col(col) == "").count()
n_distinct = df_botn.select(col).distinct().count()

print(f"=== {col} ===")
print(f"Nulls     : {n_null:,}")
print(f"Vides     : {n_empty:,}")
print(f"Distincts : {n_distinct:,}")

# Échantillon
print("\n--- Échantillon ---")
df_botn.select(col).distinct().orderBy(F.rand(42)).show(20, truncate=False)

# Doublons de casse
df_case = (
    df_botn.select(col).distinct()
    .withColumn("lower", F.lower(col))
    .groupBy("lower")
    .agg(F.count("*").alias("n"), F.collect_set(col).alias("variants"))
    .where(F.col("n") > 1)
)
print(f"Doublons de casse : {df_case.count()}")
if df_case.count() > 0:
    df_case.show(10, truncate=False)

=== route_short_name ===
Nulls     : 0
Vides     : 0
Distincts : 404

--- Échantillon ---
+-----------------+
|route_short_name |
+-----------------+
|8657             |
|22014            |
|3687             |
|091              |
|IC Notte 1955 (S)|
|EN 40416         |
|1001 (FW)        |
|EN 50462         |
|122              |
|P 269            |
|1003 (E)         |
|EC 460           |
|065 Š            |
|017 O/045 D      |
|Nattåg 94 (L)    |
|480 / 1604 / 1247|
|086 P            |
|NJ 40469         |
|52013            |
|2626             |
+-----------------+
only showing top 20 rows
Doublons de casse : 0


In [18]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNE : trip_headsign
# ═══════════════════════════════════════════════════════════

col = "trip_headsign"
total = df_botn.count()

n_null = df_botn.where(F.col(col).isNull()).count()
n_empty = df_botn.where(F.col(col) == "").count()
n_distinct = df_botn.select(col).distinct().count()

print(f"=== {col} ===")
print(f"Nulls     : {n_null:,}")
print(f"Vides     : {n_empty:,}")
print(f"Distincts : {n_distinct:,}")

# Échantillon
print("\n--- Échantillon ---")
df_botn.select("trip_id", col).distinct().orderBy(F.rand(42)).show(15, truncate=False)

# Doublons de casse
df_case = (
    df_botn.select(col).where(F.col(col).isNotNull())
    .distinct()
    .withColumn("lower", F.lower(col))
    .groupBy("lower")
    .agg(F.count("*").alias("n"), F.collect_set(col).alias("variants"))
    .where(F.col("n") > 1)
)
print(f"\nDoublons de casse : {df_case.count()}")
if df_case.count() > 0:
    df_case.show(10, truncate=False)

=== trip_headsign ===
Nulls     : 0
Vides     : 0
Distincts : 153

--- Échantillon ---
+----------------------+----------------------+
|trip_id               |trip_headsign         |
+----------------------+----------------------+
|PKP IC 83172          |Przemyśl Gł.          |
|SNCF IC Nuit 5787     |Briançon              |
|SJ 2                  |Stockholm C           |
|UZ 064                |Kharkiv               |
|ÖBB NJ 40425          |Wien Hbf              |
|FS IC Notte 1955 (S)  |Siracusa              |
|VR IC 266 (H)         |Helsinki / Helsingfors|
|HŽPP/ÖBB EN 40237     |Zagreb Gl. K.         |
|MÁV/ÖBB EN 50237      |Stuttgart Hbf         |
|CFM/CFR 402           |Chişinău              |
|TCDD/BDŽ/CFR 12503/460|București Nord        |
|MÁV/ÖBB EN 40467      |Budapest Keleti       |
|UZ 041                |Truskavets            |
|UZ 038 P              |Kyiv Pasazhyrskyi     |
|CFR IC 472            |Budapest Keleti       |
+----------------------+----------------------+
o

In [19]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNES : agency_id, agency_name, agency_timezone
# ═══════════════════════════════════════════════════════════

for col in ["agency_id", "agency_name", "agency_timezone"]:
    n_null = df_botn.where(F.col(col).isNull()).count()
    n_empty = df_botn.where(F.col(col) == "").count()
    n_distinct = df_botn.select(col).distinct().count()
    print(f"=== {col} ===")
    print(f"  Nulls: {n_null}  |  Vides: {n_empty}  |  Distincts: {n_distinct}")

# Toutes les combinaisons
print("\n--- Agences complètes ---")
(
    df_botn.select("agency_id", "agency_name", "agency_timezone")
    .distinct()
    .orderBy("agency_id")
    .show(40, truncate=False)
)

# Timezone : quels formats ?
print("\n--- Timezones distinctes ---")
df_botn.select("agency_timezone").distinct().orderBy("agency_timezone").show(20)

=== agency_id ===
  Nulls: 0  |  Vides: 0  |  Distincts: 37
=== agency_name ===
  Nulls: 0  |  Vides: 0  |  Distincts: 37
=== agency_timezone ===
  Nulls: 17  |  Vides: 0  |  Distincts: 28

--- Agences complètes ---
+-----------+-------------------------------------------------------------------------------------+----------------+
|agency_id  |agency_name                                                                          |agency_timezone |
+-----------+-------------------------------------------------------------------------------------+----------------+
|ATC        |Astra Trans Carpatic SRL                                                             |Europe/Bucharest|
|BDŽ        |БДЖ - Пътнически превози ЕООД                                                        |Europe/Sofia    |
|BDŽ        |БДЖ - Пътнически превози ЕООД                                                        |Europe/Belgrade |
|BDŽ        |БДЖ - Пътнически превози ЕООД                                        

In [20]:
# ═══════════════════════════════════════════════════════════
# CORRECTION : agency_name et agency_timezone
# ═══════════════════════════════════════════════════════════

# 1. agency_name : pour les multi-opérateurs, concaténer les noms
#    Ex: "CFM/CFR" → lookup CFM + CFR dans agencies → "Calea Ferată din Moldova / C.F.R. Călători S.A."

# Construire un mapping agency_id simple → name
ag_map = {r["agency_id"]: r["agency_name"] for r in 
    tables["agencies"].select("agency_id", "agency_name").collect()
    if r["agency_name"] is not None}

print("--- Agences multi-opérateurs à résoudre ---")
multi_agencies = (
    df_botn.where(F.col("agency_name").isNull())
    .select("agency_id").distinct()
    .collect()
)
for r in sorted(multi_agencies, key=lambda x: x[0]):
    parts = r[0].split("/")
    names = [ag_map.get(p, "?") for p in parts]
    print(f"  {r[0]:20s} → {' / '.join(names)}")

# Construire le mapping complet multi → nom concaténé
from pyspark.sql.types import StringType

def resolve_agency_name(agency_id):
    if agency_id is None:
        return None
    parts = agency_id.split("/")
    names = [ag_map.get(p, p) for p in parts]
    return " / ".join(names)

resolve_name_udf = F.udf(resolve_agency_name, StringType())

df_botn = df_botn.withColumn("agency_name",
    F.when(F.col("agency_name").isNull(), resolve_name_udf(F.col("agency_id")))
     .otherwise(F.col("agency_name"))
)

# 2. Timezone : normalisation IANA
#    CET → dépend du pays, EET idem, WET idem
#    Stratégie : mapper depuis le country de la première gare du trip

tz_by_country = {
    "FR": "Europe/Paris", "DE": "Europe/Berlin", "AT": "Europe/Vienna",
    "CH": "Europe/Zurich", "IT": "Europe/Rome", "ES": "Europe/Madrid",
    "NL": "Europe/Amsterdam", "BE": "Europe/Brussels", "CZ": "Europe/Prague",
    "PL": "Europe/Warsaw", "HU": "Europe/Budapest", "SK": "Europe/Bratislava",
    "SI": "Europe/Ljubljana", "HR": "Europe/Zagreb", "RS": "Europe/Belgrade",
    "RO": "Europe/Bucharest", "BG": "Europe/Sofia", "GR": "Europe/Athens",
    "SE": "Europe/Stockholm", "NO": "Europe/Oslo", "FI": "Europe/Helsinki",
    "DK": "Europe/Copenhagen", "EE": "Europe/Tallinn", "LT": "Europe/Vilnius",
    "LV": "Europe/Riga", "PT": "Europe/Lisbon", "IE": "Europe/Dublin",
    "GB": "Europe/London", "TR": "Europe/Istanbul", "UA": "Europe/Kyiv",
    "MD": "Europe/Chisinau", "MK": "Europe/Skopje", "ME": "Europe/Podgorica",
    "AL": "Europe/Tirane", "LU": "Europe/Luxembourg",
}

# Pour chaque trip, prendre le country du premier arrêt
w_first = Window.partitionBy("trip_id").orderBy("stop_sequence")
df_first_country = (
    df_botn
    .withColumn("rn", F.row_number().over(w_first))
    .where(F.col("rn") == 1)
    .select("trip_id", F.col("country").alias("origin_country"))
)

# Mapper country → timezone IANA
mapping_expr = F.create_map([F.lit(x) for pair in tz_by_country.items() for x in pair])

df_first_country = df_first_country.withColumn(
    "tz_iana", mapping_expr[F.col("origin_country")]
)

# Appliquer : remplacer toutes les timezones non-IANA
df_botn = (
    df_botn
    .join(df_first_country.select("trip_id", "tz_iana"), "trip_id", "left")
    .withColumn("agency_timezone",
        F.when(F.col("agency_timezone").startswith("Europe/"), F.col("agency_timezone"))
         .otherwise(F.coalesce(F.col("tz_iana"), F.col("agency_timezone")))
    )
    .drop("tz_iana")
)

# Vérifications
print(f"\nagency_name null  : {df_botn.where(F.col('agency_name').isNull()).count()}")
print(f"agency_tz null   : {df_botn.where(F.col('agency_timezone').isNull()).count()}")
print(f"\n--- Timezones après correction ---")
df_botn.select("agency_timezone").distinct().orderBy("agency_timezone").show(20)

--- Agences multi-opérateurs à résoudre ---

agency_name null  : 0
agency_tz null   : 0

--- Timezones après correction ---
+-----------------+
|  agency_timezone|
+-----------------+
| Europe/Amsterdam|
|  Europe/Belgrade|
|    Europe/Berlin|
|Europe/Bratislava|
|  Europe/Brussels|
| Europe/Bucharest|
|  Europe/Budapest|
|  Europe/Chisinau|
|Europe/Copenhagen|
|  Europe/Helsinki|
|  Europe/Istanbul|
|      Europe/Kyiv|
| Europe/Ljubljana|
|    Europe/London|
|      Europe/Oslo|
|     Europe/Paris|
| Europe/Podgorica|
|    Europe/Prague|
|      Europe/Rome|
|    Europe/Skopje|
+-----------------+
only showing top 20 rows


In [21]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNES : stop_name, stop_lat, stop_lon
# ═══════════════════════════════════════════════════════════

for col in ["stop_name", "stop_lat", "stop_lon"]:
    n_null = df_botn.where(F.col(col).isNull()).count()
    print(f"=== {col} === Nulls: {n_null}")

# stop_name : doublons de casse ?
df_case = (
    df_botn.select("stop_name").where(F.col("stop_name").isNotNull())
    .distinct()
    .withColumn("lower", F.lower("stop_name"))
    .groupBy("lower")
    .agg(F.count("*").alias("n"), F.collect_set("stop_name").alias("variants"))
    .where(F.col("n") > 1)
)
print(f"\nstop_name doublons de casse : {df_case.count()}")
if df_case.count() > 0:
    df_case.show(10, truncate=False)

# Coordonnées : plages valides Europe ?
print("\n--- Plages coordonnées ---")
df_botn.where(F.col("stop_lat").isNotNull()).select(
    F.min("stop_lat").alias("lat_min"),
    F.max("stop_lat").alias("lat_max"),
    F.min("stop_lon").alias("lon_min"),
    F.max("stop_lon").alias("lon_max"),
).show()

# Coord (0,0) ?
n_zero = df_botn.where((F.col("stop_lat") == 0) & (F.col("stop_lon") == 0)).count()
print(f"Coord (0,0) : {n_zero}")

# Gares sans coordonnées — lesquelles ?
if df_botn.where(F.col("stop_lat").isNull()).count() > 0:
    print("\n--- Gares sans coordonnées ---")
    (
        df_botn.where(F.col("stop_lat").isNull())
        .select("stop_id", "stop_name", "country")
        .distinct()
        .show(20, truncate=False)
    )

=== stop_name === Nulls: 17
=== stop_lat === Nulls: 17
=== stop_lon === Nulls: 17

stop_name doublons de casse : 0

--- Plages coordonnées ---
+---------+---------+---------+---------+
|  lat_min|  lat_max|  lon_min|  lon_max|
+---------+---------+---------+---------+
|37.068907|68.441701|-5.531908|43.106386|
+---------+---------+---------+---------+

Coord (0,0) : 0

--- Gares sans coordonnées ---
+------------------+---------+-------+
|stop_id           |stop_name|country|
+------------------+---------+-------+
|Bjala             |NULL     |NULL   |
|Ungheni Prut      |NULL     |NULL   |
|Kumarica          |NULL     |NULL   |
|Kadievo           |NULL     |NULL   |
|Siegen            |NULL     |NULL   |
|Jabalkovo         |NULL     |NULL   |
|Siegburg (Bonn)   |NULL     |NULL   |
|Cheshnegirovo     |NULL     |NULL   |
|Gelen             |NULL     |NULL   |
|Tompsan           |NULL     |NULL   |
|Liège-Guillemins  |NULL     |NULL   |
|Romcha            |NULL     |NULL   |
|Prijepolje t

In [22]:
# ═══════════════════════════════════════════════════════════
# IMPACT DES 17 LIGNES SANS COORDONNÉES
# ═══════════════════════════════════════════════════════════

# Quels trips sont concernés ?
trips_affected = (
    df_botn.where(F.col("stop_lat").isNull())
    .select("trip_id", "route_id", "agency_id", "stop_id")
    .distinct()
)
print(f"Trips concernés : {trips_affected.select('trip_id').distinct().count()}")
print(f"Routes concernées : {trips_affected.select('route_id').distinct().count()}")
trips_affected.show(20, truncate=False)

# Ces trips ont-ils des données de distance et émissions ?
print("\n--- Distance / émissions sur ces trips ---")
(
    df_botn.where(F.col("stop_lat").isNull())
    .select("trip_id", "botn_emissions_co2e", "botn_co2_per_km", "botn_distance_km")
    .distinct()
    .show(20, truncate=False)
)

# Combien d'arrêts total vs null par trip concerné ?
print("\n--- Arrêts total vs null par trip ---")
(
    df_botn
    .where(F.col("trip_id").isin([r["trip_id"] for r in trips_affected.select("trip_id").distinct().collect()]))
    .groupBy("trip_id")
    .agg(
        F.count("*").alias("total_stops"),
        F.sum(F.when(F.col("stop_lat").isNull(), 1).otherwise(0)).alias("null_stops"),
        F.first("city").alias("sample_city"),
    )
    .orderBy("trip_id")
    .show(20, truncate=False)
)

Trips concernés : 7
Routes concernées : 6
+------------------+--------+---------+------------------+
|trip_id           |route_id|agency_id|stop_id           |
+------------------+--------+---------+------------------+
|BDŽ 9647          |135     |BDŽ      |Bjala             |
|CFM/CFR 401       |22      |CFM/CFR  |Ungheni Prut      |
|BDŽ 9647          |135     |BDŽ      |Kumarica          |
|BDŽ 8656          |131     |BDŽ      |Kadievo           |
|UEX 332/1372      |256     |UEX      |Siegen            |
|UEX 333/1373      |256     |UEX      |Siegen            |
|BDŽ 8656          |131     |BDŽ      |Jabalkovo         |
|UEX 332/1372      |256     |UEX      |Siegburg (Bonn)   |
|BDŽ 8656          |131     |BDŽ      |Cheshnegirovo     |
|BDŽ 9647          |135     |BDŽ      |Gelen             |
|BDŽ 9647          |135     |BDŽ      |Tompsan           |
|ÖBB NJ 40425      |1       |ÖBB      |Liège-Guillemins  |
|BDŽ 9647          |135     |BDŽ      |Romcha            |
|ŽS/ŽPCG 1130 

In [23]:
df_botn = df_botn.filter(F.col("stop_lat").isNotNull() | F.col("stop_lon").isNotNull())

In [24]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNES : arrival_time, departure_time, stop_sequence
# ═══════════════════════════════════════════════════════════

for col in ["arrival_time", "departure_time"]:
    n_null = df_botn.where(F.col(col).isNull()).count()
    n_total = df_botn.count()
    print(f"=== {col} ===")
    print(f"  Nulls: {n_null:,} ({n_null/n_total:.1%})")
    
    # Format ?
    print(f"  Échantillon :")
    df_botn.select(col).where(F.col(col).isNotNull()).distinct().orderBy(F.rand(42)).show(10, truncate=False)

# Combinaisons null/non-null
print("\n--- Combinaisons arrival/departure ---")
df_botn.groupBy(
    F.when(F.col("arrival_time").isNull(), "null").otherwise("present").alias("arrival"),
    F.when(F.col("departure_time").isNull(), "null").otherwise("present").alias("departure"),
).count().show()

# stop_sequence
col = "stop_sequence"
n_null = df_botn.where(F.col(col).isNull()).count()
print(f"\n=== {col} ===")
print(f"  Nulls: {n_null}")
df_botn.select(F.min(col).alias("min"), F.max(col).alias("max")).show()

=== arrival_time ===
  Nulls: 2,583 (46.1%)
  Échantillon :
+------------+
|arrival_time|
+------------+
|15:19:00    |
|22:03:00    |
|12:25:00    |
|06:03:00    |
|05:40:00    |
|21:59:00    |
|09:49:00    |
|23:16:00    |
|22:52:00    |
|03:42:00    |
+------------+
only showing top 10 rows
=== departure_time ===
  Nulls: 2,230 (39.8%)
  Échantillon :
+--------------+
|departure_time|
+--------------+
|13:13:00      |
|19:03:00      |
|14:11:00      |
|22:35:00      |
|23:20:00      |
|16:14:00      |
|16:00:00      |
|22:41:00      |
|15:01:00      |
|22:15:00      |
+--------------+
only showing top 10 rows

--- Combinaisons arrival/departure ---
+-------+---------+-----+
|arrival|departure|count|
+-------+---------+-----+
|present|  present|  853|
|   null|     null|   65|
|   null|  present| 2518|
|present|     null| 2165|
+-------+---------+-----+


=== stop_sequence ===
  Nulls: 0
+---+---+
|min|max|
+---+---+
|  0| 99|
+---+---+



In [25]:
# Normaliser stop_sequence 99 → valeur réelle (dernier arrêt)
w_max = Window.partitionBy("trip_id").orderBy(F.desc("stop_sequence"))
df_botn = (
    df_botn
    .withColumn("_max_real",
        F.max(F.when(F.col("stop_sequence") != 99, F.col("stop_sequence")))
         .over(Window.partitionBy("trip_id"))
    )
    .withColumn("stop_sequence",
        F.when(F.col("stop_sequence") == 99, F.col("_max_real") + 1)
         .otherwise(F.col("stop_sequence"))
    )
    .drop("_max_real")
)

# Vérif
print(f"stop_sequence max : {df_botn.agg(F.max('stop_sequence')).collect()[0][0]}")
print(f"stop_sequence 99 restants : {df_botn.where(F.col('stop_sequence') == 99).count()}")

stop_sequence max : 67
stop_sequence 99 restants : 0


In [26]:
# ═══════════════════════════════════════════════════════════
# HARD-ENCODE country pour les 17 gares sans coordonnées
# ═══════════════════════════════════════════════════════════

# Mapping manuel basé sur les noms de gares
country_fix = {
    "Bjala": "BG",              # Bulgarie
    "Cheshnegirovo": "BG",
    "Gelen": "BG",
    "Jabalkovo": "BG",
    "Kadievo": "BG",
    "Kumarica": "BG",
    "Lukovo": "BG",
    "Romcha": "BG",
    "Tompsan": "BG",
    "Proslav": "BG",
    "Vlado Trickov": "BG",
    "Liège-Guillemins": "BE",   # Belgique
    "Prijepolje teretna": "RS", # Serbie
    "Siegburg (Bonn)": "DE",    # Allemagne
    "Siegen": "DE",
    "Ungheni Prut": "MD",       # Moldavie
}

fix_expr = F.create_map([F.lit(x) for pair in country_fix.items() for x in pair])

df_botn = (
    df_botn
    .withColumn("country",
        F.when(F.col("country").isNull(), fix_expr[F.col("stop_id")])
         .otherwise(F.col("country"))
    )
    # Recalculer timezone pour ces lignes corrigées
    .withColumn("agency_timezone",
        F.when(F.col("agency_timezone").isNull(), tz_map_expr[F.col("country")])
         .otherwise(F.col("agency_timezone"))
    )
)

print(f"country NULL : {df_botn.where(F.col('country').isNull()).count()}")
print(f"agency_timezone NULL : {df_botn.where(F.col('agency_timezone').isNull()).count()}")

# Vérif
(
    df_botn.where(F.col("stop_id").isin(list(country_fix.keys())))
    .select("stop_id", "country", "agency_timezone")
    .distinct()
    .orderBy("stop_id")
    .show(20, truncate=False)
)

country NULL : 0
agency_timezone NULL : 0
+-------+-------+---------------+
|stop_id|country|agency_timezone|
+-------+-------+---------------+
+-------+-------+---------------+



In [27]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNES : start_date, end_date
# ═══════════════════════════════════════════════════════════

for col in ["start_date", "end_date"]:
    n_null = df_botn.where(F.col(col).isNull()).count()
    n_total = df_botn.count()
    print(f"=== {col} ===")
    print(f"  Nulls: {n_null:,} ({n_null/n_total:.1%})")

# Formats ?
print("\n--- start_date échantillon ---")
df_botn.select("start_date").where(F.col("start_date").isNotNull()).distinct().orderBy("start_date").show(10, truncate=False)

print("\n--- end_date échantillon ---")
df_botn.select("end_date").where(F.col("end_date").isNotNull()).distinct().orderBy("end_date").show(10, truncate=False)

# Plages temporelles
print("\n--- Plages ---")
df_botn.where(F.col("start_date").isNotNull()).select(
    F.min("start_date").alias("start_min"),
    F.max("start_date").alias("start_max"),
    F.min("end_date").alias("end_min"),
    F.max("end_date").alias("end_max"),
).show(truncate=False)

=== start_date ===
  Nulls: 63 (1.1%)
=== end_date ===
  Nulls: 67 (1.2%)

--- start_date échantillon ---
+-------------------+
|start_date         |
+-------------------+
|2023-10-01 00:00:00|
|2023-12-10 00:00:00|
|2023-12-16 00:00:00|
|2023-12-17 00:00:00|
|2023-12-22 00:00:00|
|2023-12-23 00:00:00|
|2023-12-30 00:00:00|
|2024-03-31 00:00:00|
|2024-05-03 00:00:00|
|2024-05-08 00:00:00|
+-------------------+
only showing top 10 rows

--- end_date échantillon ---
+-------------------+
|end_date           |
+-------------------+
|2024-03-30 00:00:00|
|2024-06-12 00:00:00|
|2024-09-01 00:00:00|
|2024-09-07 00:00:00|
|2024-09-21 00:00:00|
|2024-09-22 00:00:00|
|2024-10-05 00:00:00|
|2024-11-03 00:00:00|
|2024-12-14 00:00:00|
|20240831           |
+-------------------+
only showing top 10 rows

--- Plages ---
+-------------------+---------+-------------------+--------+
|start_min          |start_max|end_min            |end_max |
+-------------------+---------+-------------------+--------+

In [28]:
# ═══════════════════════════════════════════════════════════
# NORMALISATION start_date / end_date → YYYYMMDD
# ═══════════════════════════════════════════════════════════

for col in ["start_date", "end_date"]:
    df_botn = df_botn.withColumn(col,
        F.when(F.col(col).isNull(), None)
        # Format datetime "2023-10-01 00:00:00" → "20231001"
         .when(F.col(col).rlike("^\\d{4}-\\d{2}-\\d{2}"),
            F.regexp_replace(F.substring(F.col(col), 1, 10), "-", ""))
        # Format compact "20260611" → déjà bon
         .when(F.col(col).rlike("^\\d{8}$"), F.col(col))
         .otherwise(F.col(col))
    )

# Vérif formats
print("--- start_date après normalisation ---")
df_botn.select("start_date").where(F.col("start_date").isNotNull()).distinct().orderBy("start_date").show(15, truncate=False)

print("--- end_date après normalisation ---")
df_botn.select("end_date").where(F.col("end_date").isNotNull()).distinct().orderBy("end_date").show(15, truncate=False)

# Plages
df_botn.where(F.col("start_date").isNotNull()).select(
    F.min("start_date").alias("start_min"),
    F.max("start_date").alias("start_max"),
    F.min("end_date").alias("end_min"),
    F.max("end_date").alias("end_max"),
).show(truncate=False)

--- start_date après normalisation ---
+----------+
|start_date|
+----------+
|20231001  |
|20231210  |
|20231216  |
|20231217  |
|20231222  |
|20231223  |
|20231230  |
|20240331  |
|20240503  |
|20240508  |
|20240518  |
|20240605  |
|20240607  |
|20240608  |
|20240614  |
+----------+
only showing top 15 rows
--- end_date après normalisation ---
+--------+
|end_date|
+--------+
|20240330|
|20240612|
|20240831|
|20240901|
|20240907|
|20240908|
|20240921|
|20240922|
|20241005|
|20241019|
|20241103|
|20241214|
|20250328|
|20250329|
|20250426|
+--------+
only showing top 15 rows
+---------+---------+--------+--------+
|start_min|start_max|end_min |end_max |
+---------+---------+--------+--------+
|20231001 |20260628 |20240330|20261213|
+---------+---------+--------+--------+



In [29]:
# ═══════════════════════════════════════════════════════════
# ANALYSE COLONNES : botn_distance_km, botn_emissions_co2e, botn_co2_per_km
# ═══════════════════════════════════════════════════════════

for col in ["botn_distance_km", "botn_emissions_co2e", "botn_co2_per_km"]:
    n_null = df_botn.where(F.col(col).isNull()).count()
    n_total = df_botn.count()
    n_trips_null = (
        df_botn.where(F.col(col).isNull())
        .select("trip_id").distinct().count()
    )
    print(f"=== {col} ===")
    print(f"  Nulls: {n_null:,} ({n_null/n_total:.1%}) — {n_trips_null} trips")

# Plages de valeurs
print("\n--- Statistiques ---")
df_botn.select(
    F.min("botn_distance_km").alias("dist_min_m"),
    F.max("botn_distance_km").alias("dist_max_m"),
    F.min("botn_emissions_co2e").alias("co2_min"),
    F.max("botn_emissions_co2e").alias("co2_max"),
    F.min("botn_co2_per_km").alias("co2km_min"),
    F.max("botn_co2_per_km").alias("co2km_max"),
).show(truncate=False)

# Trips sans distance
print("\n--- Trips sans distance ---")
(
    df_botn.where(F.col("botn_distance_km").isNull())
    .select("trip_id", "agency_id", "botn_emissions_co2e", "botn_co2_per_km")
    .distinct()
    .orderBy("agency_id", "trip_id")
    .show(30, truncate=False)
)

=== botn_distance_km ===
  Nulls: 593 (10.6%) — 42 trips
=== botn_emissions_co2e ===
  Nulls: 1,415 (25.3%) — 100 trips
=== botn_co2_per_km ===
  Nulls: 1,415 (25.3%) — 100 trips

--- Statistiques ---
+----------+----------+-------+-------+-----------+-----------+
|dist_min_m|dist_max_m|co2_min|co2_max|co2km_min  |co2km_max  |
+----------+----------+-------+-------+-----------+-----------+
|405.0     |1847.0    |0.16   |30.5   |0.306122449|38.82352941|
+----------+----------+-------+-------+-----------+-----------+


--- Trips sans distance ---
+----------------------+---------+-------------------+---------------+
|trip_id               |agency_id|botn_emissions_co2e|botn_co2_per_km|
+----------------------+---------+-------------------+---------------+
|OTE 141               |OTE      |NULL               |NULL           |
|OTE 154               |OTE      |NULL               |NULL           |
|PKP IC 38172          |PKP      |NULL               |NULL           |
|PKP IC 83172          

In [30]:
# ═══════════════════════════════════════════════════════════
# CALCUL DISTANCE SEGMENT ENTRE ARRÊTS CONSÉCUTIFS
# ═══════════════════════════════════════════════════════════

from math import radians, sin, cos, sqrt, atan2
from pyspark.sql.types import DoubleType

CORRECTION_FACTOR = 1.3

# Distance entre arrêts consécutifs via Haversine
w_prev = Window.partitionBy("trip_id").orderBy("stop_sequence")

# Calculer la distance au prochain arrêt pour chaque ligne
df_botn = (
    df_botn
    .withColumn("next_lat", F.lead("stop_lat").over(w_prev))
    .withColumn("next_lon", F.lead("stop_lon").over(w_prev))
)

# Haversine en Spark natif (pas de UDF, plus performant)
df_botn = (
    df_botn
    .withColumn("dlat", F.radians(F.col("next_lat") - F.col("stop_lat")))
    .withColumn("dlon", F.radians(F.col("next_lon") - F.col("stop_lon")))
    .withColumn("a",
        F.sin(F.col("dlat") / 2) ** 2 +
        F.cos(F.radians(F.col("stop_lat"))) *
        F.cos(F.radians(F.col("next_lat"))) *
        F.sin(F.col("dlon") / 2) ** 2
    )
    .withColumn("segment_dist_m",
        F.when(
            F.col("next_lat").isNotNull(),
            2 * 6371000 * F.asin(F.sqrt(F.col("a"))) * CORRECTION_FACTOR
        ).otherwise(F.lit(None))
    )
    .drop("next_lat", "next_lon", "dlat", "dlon", "a")
)

# Vérification
n_with_segment = df_botn.where(F.col("segment_dist_m").isNotNull()).count()
n_total = df_botn.count()

print(f"Lignes avec segment_dist_m (arrêts non-terminaux) : {n_with_segment:,} / {n_total:,}")

# Statistiques sur les distances de segments
print("\n--- Statistiques segment_dist_m (km) ---")
df_botn.select(
    (F.min("segment_dist_m") / 1000).alias("min_km"),
    (F.max("segment_dist_m") / 1000).alias("max_km"),
    (F.avg("segment_dist_m") / 1000).alias("avg_km"),
    (F.percentile_approx(F.col("segment_dist_m") / 1000, 0.5)).alias("median_km"),
).show()


Lignes avec segment_dist_m (arrêts non-terminaux) : 5,193 / 5,601

--- Statistiques segment_dist_m (km) ---
+------+-----------------+-----------------+------------------+
|min_km|           max_km|           avg_km|         median_km|
+------+-----------------+-----------------+------------------+
|   0.0|1737.083574426228|79.76006419733139|14.729833278372615|
+------+-----------------+-----------------+------------------+



In [31]:
# Dédoubler le df final
n_before = df_botn.count()
df_botn = df_botn.dropDuplicates(["trip_id", "stop_id", "stop_sequence"])
n_after = df_botn.count()
print(f"\nDédup final : {n_before:,} → {n_after:,} ({n_before - n_after:,} doublons)")
print(f"Trips : {df_botn.select('trip_id').distinct().count()}")

# Vérif : encore des doublons ?
df_dup_check = (
    df_botn.groupBy("trip_id", "stop_sequence")
    .count().where(F.col("count") > 1)
)
print(f"Doublons restants : {df_dup_check.count()}")


Dédup final : 5,601 → 3,265 (2,336 doublons)
Trips : 408
Doublons restants : 6


In [32]:
# ═══════════════════════════════════════════════════════════
# 1. Les 6 doublons restants
# ═══════════════════════════════════════════════════════════

print("--- Doublons (trip_id, stop_sequence) ---")
(
    df_botn.groupBy("trip_id", "stop_sequence")
    .count().where(F.col("count") > 1)
    .orderBy("trip_id", "stop_sequence")
    .show(10, truncate=False)
)

# Voir les lignes en question
df_dup_ids = (
    df_botn.groupBy("trip_id", "stop_sequence")
    .count().where(F.col("count") > 1)
    .select("trip_id", "stop_sequence")
)

(
    df_botn.join(df_dup_ids, ["trip_id", "stop_sequence"], "inner")
    .select("trip_id", "stop_sequence", "stop_id", "stop_name", "arrival_time", "departure_time")
    .orderBy("trip_id", "stop_sequence")
    .show(20, truncate=False)
)

--- Doublons (trip_id, stop_sequence) ---
+-------------+-------------+-----+
|trip_id      |stop_sequence|count|
+-------------+-------------+-----+
|PKP/ČD EC 460|1            |2    |
|VR IC 274    |3            |2    |
|VR IC 274    |4            |2    |
|VR IC 274    |6            |2    |
|ÖBB NJ 40235 |66           |3    |
|ÖBB NJ 40295 |66           |3    |
+-------------+-------------+-----+

+-------------+-------------+----------------------+----------------------+------------+--------------+
|trip_id      |stop_sequence|stop_id               |stop_name             |arrival_time|departure_time|
+-------------+-------------+----------------------+----------------------+------------+--------------+
|PKP/ČD EC 460|1            |Pardubice hl.n.       |Pardubice hl.n.       |NULL        |18:54:00      |
|PKP/ČD EC 460|1            |Jarosław              |Jarosław              |NULL        |17:18:00      |
|VR IC 274    |3            |Ylivieska             |Ylivieska             |NU

In [33]:
# Trips avec doublons
trips_with_dup = [r[0] for r in
    df_botn.groupBy("trip_id", "stop_sequence")
    .count().where(F.col("count") > 1)
    .select("trip_id").distinct().collect()
]
print(f"Trips à renuméroter : {trips_with_dup}")

# Renuméroter ces trips entièrement par horaire
w_renum = Window.partitionBy("trip_id").orderBy(
    F.coalesce("departure_time", "arrival_time")
)

df_botn = (
    df_botn
    .withColumn("stop_sequence",
        F.when(
            F.col("trip_id").isin(trips_with_dup),
            F.row_number().over(w_renum) - 1
        ).otherwise(F.col("stop_sequence"))
    )
)

n_dup = df_botn.groupBy("trip_id", "stop_sequence").count().where(F.col("count") > 1).count()
print(f"Doublons restants : {n_dup}")

Trips à renuméroter : ['PKP/ČD EC 460', 'VR IC 274', 'ÖBB NJ 40235', 'ÖBB NJ 40295']
Doublons restants : 0


In [34]:
# ═══════════════════════════════════════════════════════════
# ANALYSE : co2_per_km existants par pays et opérateur
# ═══════════════════════════════════════════════════════════

# 1. Valeurs par opérateur (premier opérateur du agency_id)
df_co2_ref = (
    df_botn
    .where(F.col("botn_co2_per_km").isNotNull())
    .withColumn("main_agency", F.split("agency_id", "/")[0])
    .select("trip_id", "main_agency", "country", "botn_co2_per_km", "botn_distance_km")
    .distinct()
)

print("--- co2_per_km par opérateur principal ---")
(
    df_co2_ref
    .groupBy("main_agency")
    .agg(
        F.count("*").alias("n_trips"),
        F.round(F.min("botn_co2_per_km"), 2).alias("min"),
        F.round(F.avg("botn_co2_per_km"), 2).alias("avg"),
        F.round(F.max("botn_co2_per_km"), 2).alias("max"),
    )
    .orderBy("main_agency")
    .show(30, truncate=False)
)

# 2. Moyenne globale par pays d'origine du trip
w_first = Window.partitionBy("trip_id").orderBy("stop_sequence")
df_trip_origin = (
    df_botn
    .withColumn("rn", F.row_number().over(w_first))
    .where(F.col("rn") == 1)
    .select("trip_id", F.col("country").alias("origin_country"))
)

print("\n--- co2_per_km par pays d'origine ---")
(
    df_co2_ref
    .join(df_trip_origin, "trip_id")
    .groupBy("origin_country")
    .agg(
        F.count("*").alias("n_trips"),
        F.round(F.avg("botn_co2_per_km"), 2).alias("avg_co2_per_km"),
    )
    .orderBy("origin_country")
    .show(30, truncate=False)
)

# 3. Les trips sans émissions — quels opérateurs/pays ?
print("\n--- Trips SANS co2_per_km ---")
(
    df_botn
    .where(F.col("botn_co2_per_km").isNull())
    .select("trip_id", "agency_id")
    .distinct()
    .withColumn("main_agency", F.split("agency_id", "/")[0])
    .join(df_trip_origin, "trip_id")
    .groupBy("main_agency", "origin_country")
    .agg(F.count("*").alias("n_trips"))
    .orderBy("main_agency", "origin_country")
    .show(30, truncate=False)
)

# 4. Moyenne globale tous opérateurs confondus
avg_global = df_co2_ref.select(F.avg("botn_co2_per_km")).collect()[0][0]
print(f"\nMoyenne globale co2_per_km : {avg_global:.2f} g/km")

--- co2_per_km par opérateur principal ---
+-----------+-------+-----+-----+-----+
|main_agency|n_trips|min  |avg  |max  |
+-----------+-------+-----+-----+-----+
|ATC        |2      |23.18|23.18|23.18|
|BDŽ        |15     |20.41|20.94|21.44|
|CFM        |6      |23.12|23.12|23.12|
|CFR        |40     |16.31|21.77|25.58|
|CS         |10     |20.64|21.84|22.83|
|ES         |30     |14.16|15.64|19.01|
|FS         |37     |18.61|20.15|21.52|
|GA         |2      |0.31 |0.31 |0.31 |
|GWR        |2      |20.45|20.45|20.45|
|HŽPP       |22     |5.87 |11.34|27.36|
|MÁV        |8      |17.09|17.09|17.09|
|PKP        |28     |20.84|26.71|38.82|
|RJ         |10     |17.56|21.67|27.84|
|SJ         |20     |0.96 |4.08 |11.19|
|SJN        |4      |0.32 |0.32 |0.32 |
|SNCF       |18     |2.78 |3.39 |6.48 |
|ST         |18     |1.11 |11.19|16.46|
|TCDD       |24     |21.16|22.68|24.22|
|UZ         |111    |12.47|17.97|30.25|
|VR         |8      |12.57|13.53|14.05|
|VY         |2      |0.33 |0.33 |0.33

In [35]:
# ═══════════════════════════════════════════════════════════
# ENRICHISSEMENT : co2_per_km par opérateur → pays → global
# CALCUL ÉMISSIONS PAR SEGMENT
# ═══════════════════════════════════════════════════════════

# 1. Moyenne par opérateur principal (depuis les trips connus)
df_avg_agency = (
    df_botn
    .where(F.col("botn_co2_per_km").isNotNull())
    .withColumn("main_agency", F.split("agency_id", "/")[0])
    .select("trip_id", "main_agency", "botn_co2_per_km")
    .distinct()
    .groupBy("main_agency")
    .agg(F.avg("botn_co2_per_km").alias("avg_co2_agency"))
)

# 2. Moyenne par pays d'origine
w_first = Window.partitionBy("trip_id").orderBy("stop_sequence")
df_trip_origin = (
    df_botn
    .withColumn("rn", F.row_number().over(w_first))
    .where(F.col("rn") == 1)
    .select("trip_id", F.col("country").alias("origin_country"))
)

df_avg_country = (
    df_botn
    .where(F.col("botn_co2_per_km").isNotNull())
    .select("trip_id", "botn_co2_per_km").distinct()
    .join(df_trip_origin, "trip_id")
    .groupBy("origin_country")
    .agg(F.avg("botn_co2_per_km").alias("avg_co2_country"))
)

# 3. Appliquer : opérateur → pays → global (17.22)
df_botn = (
    df_botn
    .withColumn("main_agency", F.split("agency_id", "/")[0])
    .join(df_trip_origin, "trip_id", "left")
    .join(df_avg_agency, "main_agency", "left")
    .join(df_avg_country, "origin_country", "left")
    .withColumn("botn_co2_per_km",
        F.coalesce(
            F.col("botn_co2_per_km"),       # valeur originale
            F.col("avg_co2_agency"),         # fallback opérateur
            F.col("avg_co2_country"),        # fallback pays
            F.lit(17.22)                     # fallback global (g CO2e/km)
        )
    )
    # Calculer émissions par segment = co2_per_km × segment_dist_km
    .withColumn("segment_emissions_co2e",
        F.when(
            F.col("segment_dist_m").isNotNull(),
            F.round(F.col("botn_co2_per_km") * F.col("segment_dist_m") / 1000000, 2)
        ).otherwise(F.lit(None))
    )
    .drop("main_agency", "origin_country", "avg_co2_agency", "avg_co2_country")
)

# Vérif
n_co2_null = df_botn.where(F.col("botn_co2_per_km").isNull()).count()
n_emi_null = df_botn.where(F.col("segment_emissions_co2e").isNull()).count()
n_emi_with_value = df_botn.where(F.col("segment_emissions_co2e").isNotNull()).count()

print(f"botn_co2_per_km NULL : {n_co2_null}")
print(f"segment_emissions_co2e NULL : {n_emi_null} (normal pour derniers arrêts)")
print(f"segment_emissions_co2e avec valeur : {n_emi_with_value}")

# Statistiques sur les émissions par segment
print("\n--- Statistiques segment_emissions_co2e (kg CO2e) ---")
df_botn.select(
    F.min("segment_emissions_co2e").alias("min_kg"),
    F.max("segment_emissions_co2e").alias("max_kg"),
    F.avg("segment_emissions_co2e").alias("avg_kg"),
    F.percentile_approx("segment_emissions_co2e", 0.5).alias("median_kg"),
).show()

# Exemples
print("\n--- Exemples (trip_id, segment, distance_km, emissions_kg) ---")
(
    df_botn
    .where(F.col("segment_dist_m").isNotNull())
    .select(
        "trip_id", "stop_sequence", "stop_name",
        F.round(F.col("segment_dist_m") / 1000, 1).alias("dist_km"),
        "segment_emissions_co2e",
        F.col("botn_co2_per_km").alias("co2_per_km")
    )
    .orderBy("trip_id", "stop_sequence")
    .show(20, truncate=False)
)


botn_co2_per_km NULL : 0
segment_emissions_co2e NULL : 295 (normal pour derniers arrêts)
segment_emissions_co2e avec valeur : 2970

--- Statistiques segment_emissions_co2e (kg CO2e) ---
+------+------+------------------+---------+
|min_kg|max_kg|            avg_kg|median_kg|
+------+------+------------------+---------+
|   0.0| 53.74|1.8327542087542132|     0.84|
+------+------+------------------+---------+


--- Exemples (trip_id, segment, distance_km, emissions_kg) ---
+------------+-------------+--------------+-------+----------------------+-----------+
|trip_id     |stop_sequence|stop_name     |dist_km|segment_emissions_co2e|co2_per_km |
+------------+-------------+--------------+-------+----------------------+-----------+
|ATC IR 11500|0            |Arad          |0.0    |0.0                   |23.17880795|
|ATC IR 11500|1            |Timişoara Nord|70.3   |1.63                  |23.17880795|
|ATC IR 11500|2            |Lugoj         |48.2   |1.12                  |23.17880795|
|A

In [36]:
# ═══════════════════════════════════════════════════════════
# RÉCAPITULATIF NETTOYAGE BOTN
# ═══════════════════════════════════════════════════════════

total = df_botn.count()
print(f"df_botn final : {total:,} lignes")
print(f"Trips : {df_botn.select('trip_id').distinct().count()}")
print(f"Gares : {df_botn.select('stop_id').distinct().count()}")
print(f"Routes : {df_botn.select('route_id').distinct().count()}")
print(f"Agences : {df_botn.select('agency_id').distinct().count()}")
print(f"Pays : {df_botn.select('country').distinct().count()}")

print(f"\n--- Nulls par colonne ---")
for c in df_botn.columns:
    n = df_botn.where(F.col(c).isNull()).count()
    if n > 0:
        print(f"  {c:25s}: {n:>5,} ({n/total:.1%})")

df_botn.show(30, truncate=False)

df_botn final : 3,265 lignes
Trips : 408
Gares : 679
Routes : 204
Agences : 37
Pays : 26

--- Nulls par colonne ---
  service_id               :    38 (1.2%)
  route_long_name          : 3,265 (100.0%)
  parent_station           : 3,265 (100.0%)
  arrival_time             : 1,453 (44.5%)
  departure_time           : 1,250 (38.3%)
  start_date               :    42 (1.3%)
  end_date                 :    46 (1.4%)
  botn_distance_km         :   422 (12.9%)
  botn_emissions_co2e      :   910 (27.9%)
  city                     :     9 (0.3%)
  segment_dist_m           :   295 (9.0%)
  segment_emissions_co2e   :   295 (9.0%)
+------------+-----------------+-------------------+--------+-----------+----------+----------------+---------------+--------------+---------------+---------+-----------------------------+----------------+---------------+----------------+----------------+--------------+------------+--------------+-------------+----------+--------+----------------+------------+-------+--

In [37]:
# Sauvegarde parquet
output_path = "./processed/botn_cleaned.parquet"
df_botn.write.mode("overwrite").parquet(output_path)
print(f"\nDataset nettoyé sauvegardé : {output_path}")


Dataset nettoyé sauvegardé : ./processed/botn_cleaned.parquet
